# Gauss-Quadrature

$$
\int_{a}^{b} \phi(t) dt = \sum_{i=1}^{m} \Phi(t_i) \alpha_i
$$

$$ 
\alpha_i = \int_{a}^{b} w(x) \prod_{j=1, j\neq i}^{m} \frac{t-t_j}{t_i-t_j} dt 
$$

## Gauss-Legendre Integration
- Integrate on $[-1,1]$
- $w(t) = 1$

$$
\int_{-1}^{1} f(x) dx = \sum_{i=1}^{m} f(x_i) \cdot \alpha_i
$$

Hereby the quadrature nodes are predefined.
See https://rosettacode.org/wiki/Numerical_integration/Gauss-Legendre_Quadrature#Python 
for reference.

## Variable limits
$$
\int_a^b f(t) dt = \frac{b-a}{2} \int_{-1}^{1} f\left(\frac{b-a}{2} x + \frac{a+b}{2}\right) dx = \frac{b-a}{2} \sum_{i=1}^{m} f\left(\frac{b-a}{2} x_i + \frac{a+b}{2}\right) \cdot \alpha_i
$$

In [31]:
import numpy as np

# Testfunction 
f = lambda x:  5.0 * (x-0.5) * np.exp(0.25 * (x-0.5)**2)
F = lambda x: 10.0 * np.exp(0.25 * (x-0.5)**2)


# Quadrature Nodes and Weights
t_i = [-np.sqrt(3./5.), 0.0, np.sqrt(3./5.)]
a_i = [5./9., 8./9., 5./9.]

'''
--------------------------
Integral over [-1,1]
--------------------------
'''
F_int_ana = F(1) - F(-1)
print("Analytic integral over [-1, 1]: %e" % (F_int_ana))

# Gauss-Legendre-Integration
F_int_GL = 0.0
for i in range(0,len(t_i)):
    F_int_GL += f(t_i[i]) * a_i[i]

print("Gauss-Legendre integral over [-1, 1]: %e" % (F_int_GL))


'''
--------------------------
Integral over [a,b]
--------------------------
'''
a = 3.315
b = 3.54
F_int_ana = F(b) - F(a)
print("Analytic integral over [%d, %d]: %e" % (a, b, F_int_ana))

# Gauss-Legendre-Integration
F_int_GL = 0.0
for i in range(0,len(t_i)):
    xi = ((b-a) / 2.0) * t_i[i] + (a+b)/2.0
    F_int_GL += f(xi) * a_i[i]
F_int_GL *= (b-a)/2.0
print("Gauss-Legendre integral over [-1, 1]: %e" % (F_int_GL))

Analytic integral over [-1, 1]: -6.905602e+00
Gauss-Legendre integral over [-1, 1]: -6.902713e+00
Analytic integral over [3, 3]: 2.828058e+01
Gauss-Legendre integral over [-1, 1]: 2.828058e+01


In [62]:
from numpy import *
 
##################################################################
# Recursive generation of the Legendre polynomial of order n
def Legendre(n,x):
    x=array(x)
    if (n==0):
        return x*0+1.0
    elif (n==1):
        return x
    else:
        return ((2.0*n-1.0)*x*Legendre(n-1,x)-(n-1)*Legendre(n-2,x))/n
    
    
##################################################################
# Derivative of the Legendre polynomials
def DLegendre(n,x):
    x=array(x)
    if (n==0):
        return x*0
    elif (n==1):
        return x*0+1.0
    else:
        return (n/(x**2-1.0))*(x*Legendre(n,x)-Legendre(n-1,x))
    
    
##################################################################
# Roots of the polynomial obtained using Newton-Raphson method
def LegendreRoots(polyorder,tolerance=1e-20):
    if polyorder<2:
        err=1 # bad polyorder no roots can be found
    else:
        roots=[]
        # The polynomials are alternately even and odd functions. So we evaluate only half the number of roots. 
        for i in range(1, int(int(polyorder) / 2) +1):
            x=cos(pi*(i-0.25)/(polyorder+0.5))
            error=10*tolerance
            iters=0
            while (error>tolerance) and (iters<1000):
                dx=-Legendre(polyorder,x)/DLegendre(polyorder,x)
                x=x+dx
                iters=iters+1
                error=abs(dx)
            roots.append(x)
        # Use symmetry to get the other roots
        roots=array(roots)
        if polyorder%2==0:
            roots=concatenate( (-1.0*roots, roots[::-1]) )
        else:
            roots=concatenate( (-1.0*roots, [0.0], roots[::-1]) )
        err=0 # successfully determined roots
    return [roots, err]


##################################################################
# Weight coefficients
def GaussLegendreWeights(polyorder):
    W=[]
    [xis,err]=LegendreRoots(polyorder)
    if err==0:
        W=2.0/( (1.0-xis**2)*(DLegendre(polyorder,xis)**2) )
        err=0
    else:
        err=1 # could not determine roots - so no weights
    return [W, xis, err]

##################################################################
# The integral value 
# func : the integrand
# a, b : lower and upper limits of the integral
# polyorder: order of the Legendre polynomial to be used
#
def GaussLegendreQuadrature(func, polyorder, a, b):
    [Ws,xs, err]= GaussLegendreWeights(polyorder)
    if err==0:
        ans=(b-a)*0.5*sum( Ws*func( (b-a)*0.5*xs+ (b+a)*0.5 ) )
    else: 
        # (in case of error)
        err=1
        ans=None
    return [ans,err]



##################################################################
# The integrand - change as required
def func(x):
    return exp(x)
##################################################################
#
order=5
a = 0
b = 2.0
[Ws,xs,err]=GaussLegendreWeights(order)
if err==0:
    print("Order    : ", order)
    print("Roots    : ", xs)
    print("Weights  : ", Ws)
else:
    print("Roots/Weights evaluation failed")
    
    
# Integrating the function
[ans,err]=GaussLegendreQuadrature(func , order, a, b)
if err==0:
    print("Integral : ", ans)
else:
    print("Integral evaluation failed")

Order    :  5
Roots    :  [-0.90617985 -0.53846931  0.          0.53846931  0.90617985]
Weights  :  [0.23692689 0.47862867 0.56888889 0.47862867 0.23692689]
Integral :  6.389056096688674


# Gauss-Legendre-Quadrature

References:
- Fast and Accurate Computation of Gauss-Legendre and Gauss-Jacobi Quadrature Nodes and Weights, Nicholas Hale and Alex Townsend, OCCAM Preprint Number 12/79
- https://rosettacode.org/wiki/Numerical_integration/Gauss-Legendre_Quadrature


## General formulation
The integral of $f(x)$ is approximated over the interval $[-1,1]$ by a polynomial function $g(x)$ and a known weighting function $W(x)$.
$$
\int_{-1}^{1} f(x) dx = \int_{-1}^{1} W(x) g(x) dx
$$

Then $g(x)$ and $W(x)$ are approximated by a sum of function values at specified points $x_i$, multiplied by some weights $w_i$
$$
\int_{-1}^{1} W(x) g(x) dx \approx \sum_{i=1}^{n} w_i g(x_i)
$$

## Gauss-Legendre formulation
In the case o Gauss-Legendre formulation, the weighting function $W(x) = 1$, which leads to
$$
\int_{-1}^{1} f(x) dx \approx \sum_{i=1}^{n} w_i f(x_i)
$$

### How to obtain the weights $w_i$ and nodes $x_i$
The $n$ evaluation points $x_i$ for a n-point rule (called "nodes") ar roots of n-th order Legendre Polynomials $P_n(x)$.
These Legendre-Polynomials are defined through the following recursive rule:
$$
P_0(x) = 1
$$
$$
P_1(x) = x
$$
$$
n P_n(x) = (2n-1) x P_{n-1}(x) - (n-1) P_{n-2}(x)
$$

The recursive rule for the Legendre-polynom derivative is
$$
P_n'(x) = \frac{n}{x^2-1} (xP_n(x) - P_{n-1}(x))
$$


The roots can be computed numerically by the Newton-Raphson mathod:
$$
x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)}
$$

As first guess $x_0$ for the i-th root of a n-order polynomial $P_n$ choose
$$
x_0 = \cos\left(\pi \frac{i-\frac{1}{4}}{n+\frac{1}{2}} \right)
$$

After the determination of the nodes $x_i$, the weights can be approximated through
$$
w_i = \frac{2}{\left(1-x_i^2\right) \left[P_n'(x_i)\right]^2}
$$

Finally after obtaining nodes $x_i$ and weights $w_i$, it is possible to approximate the integral of a function
$f(x)$ over the interval $[a,b]$ by
$$
\int_{a}^{b} f(x) dx \approx \frac{b-a}{2} \sum_{i=1}^{n} w_i f(t_i)
$$ 
with
$$
t_i = \frac{b-a}{2} x_i + \frac{a+b}{2}
$$

## Gauss-Radau formulation
Gauss-Radau is a variation on the Gauss-Legendre quadarature rule, where the first the endpoint of the interval is also used. 

The Gauss-Radau  nodes $x_i$ satisfy for $i=1,..., n$
$$
\frac{P_{n-1} (x_i) + P_n(x_i)}{1+x_i} = 0
$$

and the weights $w_i$ satisfy for $i = 1,...,n-1$
$$ 
w_i = \frac{1}{(1-x_i)\left[P_{n-1}'(x_i)\right]^2}
$$
and for $i=n$
$$
w_i = \frac{2}{n^2}
$$

The roots can also be found using Newton-Raphson method:
$$
x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)}
$$
Hereby $f(x_i)$ is defined as 
$$
f(x_i) = \frac{P_{n-1} (x_i) + P_n(x_i)}{1+x_i}
$$
and $f'(x_i)$ is defined as
$$
f'(x_i) = \frac{P'_{n-1} (x_i) + P'_n(x_i)}{1+x_i} - \frac{P_{n-1} (x_i) + P_n(x_i)}{(1+x_i)^{1/2}} 
$$


## Algorithm to compute roots and weights
1. Define recursive functions to compute Legendre polynomial $P_n(x_i)$ and Legendre polynomial derivative $P_n'(x_i)$
2. Compute first guess of root $x_i$ 
3. Perform Newtpn-Raphson-Iterations for respective root-function until certain limit is reached
4. Compute weight for respective roots





In [18]:
from numpy import *


'''
------------------------------------------------------------------
Function: Evaluates the Legendre-Polynom P_n(x) of order n
          at point x
------------------------------------------------------------------
'''
def Legendre(n,x):
    x=array(x)
    if (n==0):
        return 1.0
    elif (n==1):
        return x
    else:
        return ((2.0*n-1.0)*x*Legendre(n-1,x)-(n-1)*Legendre(n-2,x))/n
    
    
'''
------------------------------------------------------------------
Function: Evaluates the derivative of the Legendre-Polynom P_n(x) 
          of order n at point x
------------------------------------------------------------------
'''
def DLegendre(n,x):
    x=array(x)
    if (n==0):
        return 0
    elif (n==1):
        return 1.0
    else:
        return (n/(x**2-1.0))*(x*Legendre(n,x)-Legendre(n-1,x))
    
    
'''
------------------------------------------------------------------
Function: Calcuate the Gauss-Radau roots using 
          Newton-Raphson method
------------------------------------------------------------------
'''
def RadauRoots(n, tolerance=1e-20):
    roots = []
    if n < 2:
        return roots # bad polyorder - no roots can be found
    else:
        # First root is 1
        roots. append(1)
        # The remaining roots are computed through newtpn-raphson
        for i in range(1, n):
            x = cos(pi*(i-0.25)/(n+0.5))
            error = 10*tolerance
            iters = 0
            iiters = 0
            while (error>tolerance) and (iters<5000):
                while (1.+x < 0.0) and (iiters<1000):
                    x += 1. / (n-1)
                f    =  (Legendre(n-1,x) + Legendre(n,x)) / (1.+ x)
                df_1 =  (DLegendre(n-1,x) + DLegendre(n,x)) / (1.+ x)
                df_2 = -(Legendre(n-1,x) +  Legendre(n,x)) / (1.+ x)**0.5
                df = df_1 + df_2
                delta = -f / df
                x = x + delta
                iters=iters+1
                error=abs(delta)
            roots.append(x)
        
    # Return roots in ascending order
    return roots[-1::-1]


'''
------------------------------------------------------------------
Function: Calcuate the Gauss-Radau weights w_i
------------------------------------------------------------------
'''
def GaussRadauWeights(n):
    w=[]
    roots = RadauRoots(n)
    if len(roots) > 0:
        for i, ri in enumerate(roots):
            if i < len(roots)-1:
                wi = 1.0 / ( (1.0 - ri) * DLegendre(n-1, ri)**2 )
            else:
                wi = 2. / (n*n)
            w.append(wi)
    else:
        print("Error: Could not compute Gauss-Radau weights")
        print("No roots available")
        
    return [w, roots]

